In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json

# Import API key
from config import gkey

In [2]:
# Import cities file into the cities_pd DataFrame
file_to_load = ("Resources/cities.csv")
cities_df = pd.read_csv(file_to_load)
cities_df

,City,State
0,New York City,New York
1,Los Angeles,California
2,Chicago,Illinois
3,Houston,Texas
4,Philadelphia,Pennsylvania
...,...,...
95,Fremont,California
96,Baton Rouge,Louisiana
97,Richmond,Virginia
98,Boise,Idaho


In [3]:
# Add columns for lat, lng, airport name, airport address, airport rating.
# Hint: Be sure to specify "" as the initial value for each column.
cities_df["Lat"] = ""
cities_df['Lng'] = ""
cities_df["Airport Name"] = ""
cities_df["Airport Address"] = ""
cities_df["Airport Rating"] = ""
cities_df

,City,State,Lat,Lng,Airport Name,Airport Address,Airport Rating
0,New York City,New York,,,,,
1,Los Angeles,California,,,,,
2,Chicago,Illinois,,,,,
3,Houston,Texas,,,,,
4,Philadelphia,Pennsylvania,,,,,
...,...,...,...,...,...,...,...
95,Fremont,California,,,,,
96,Baton Rouge,Louisiana,,,,,
97,Richmond,Virginia,,,,,
98,Boise,Idaho,,,,,


In [4]:
# Loop through the cities_pd and get the lat/long for each city
# Hint: `requests.get(target_url).json()`
# Hint: `for index, row in cities_pd.iterrows():`
# Hint: `cities_pd.loc`

#establish url and parameters
url = 'https://maps.googleapis.com/maps/api/geocode/json?' 
params = {'key':gkey}

#Iterate through the dataframe
for index, row in cities_df.iterrows():
    #define city from row column
    city = row['City']
    #add address to params
    params['address'] = city
    
    cities = requests.get(url, params=params).json()
    cities_df.loc[index, "Lat"] = cities['results'][0]['geometry']['location']['lat']
    cities_df.loc[index, "Lng"] = cities['results'][0]['geometry']['location']['lng']
cities_df

,City,State,Lat,Lng,Airport Name,Airport Address,Airport Rating
0,New York City,New York,40.712775,-74.005973,,,
1,Los Angeles,California,34.052234,-118.243685,,,
2,Chicago,Illinois,41.878114,-87.629798,,,
3,Houston,Texas,29.760427,-95.369803,,,
4,Philadelphia,Pennsylvania,39.952584,-75.165222,,,
...,...,...,...,...,...,...,...
95,Fremont,California,37.54854,-121.988583,,,
96,Baton Rouge,Louisiana,30.451468,-91.187147,,,
97,Richmond,Virginia,37.540725,-77.436048,,,
98,Boise,Idaho,43.615019,-116.202314,,,


In [6]:
# Use the lat/lng we recovered to identify airports
# Hint: Code is similar to above block (i.e. you will need to use cities_pd.iterrows() again)
# Hint: Use the keyword search for "international+airport"
# Hint: Use a try-except to handle airports that are missing ratings

# params dictionary to update each iteration
params = {
    "radius": 8000,
    "types": "airport",
    "keyword": "international airport",
    "key": gkey
}

# Use the lat/lng we recovered to identify airports
for index, row in cities_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        cities_df.loc[index, "Airport Name"] = name_address["results"][0]["name"]
        cities_df.loc[index, "Airport Address"] = name_address["results"][0]["vicinity"]
        cities_df.loc[index, "Airport Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


In [8]:
# Save Data to csv
cities_df.to_csv("Airport_Output.csv")

# Visualize to confirm airport data appears
cities_df.head(10)

,City,State,Lat,Lng,Airport Name,Airport Address,Airport Rating
0,New York City,New York,40.712775,-74.005973,TSS Heliport,"E 34th St, New York",4.4
1,Los Angeles,California,34.052234,-118.243685,ABC-TV Heliport,Los Angeles,0
2,Chicago,Illinois,41.878114,-87.629798,MAG USA,"100 N LaSalle St, Chicago",5
3,Houston,Texas,29.760427,-95.369803,George Bush Intercontinental Airport,"2800 N Terminal Rd, Houston",4.2
4,Philadelphia,Pennsylvania,39.952584,-75.165222,Baggage Claim E (PHL),"Airport Terminals E & F ,Philadelphia Internat...",3.9
5,Phoenix,Arizona,33.448377,-112.074037,Phoenix Sky Harbor International Airport,"3400 E Sky Harbor Blvd, Phoenix",4.3
6,San Antonio,Texas,29.424122,-98.493628,Lackland Air Force Airport,San Antonio,0
7,San Diego,California,32.715738,-117.161084,San Diego International Airport,"3225 N Harbor Dr, San Diego",4.3
8,Dallas,Texas,32.776664,-96.796988,Dallas Love Field Airport,"8008 Herb Kelleher Way, Dallas",4.5
9,San Jose,California,37.338208,-121.886329,Norman Y. Mineta San Jose International Airport,"1701 Airport Blvd, San Jose",4.3
